In [21]:
import cv2
import numpy as np
import pytesseract
from PyPDF2 import PdfReader
import tempfile
import os

In [14]:
def extract_images_from_pdf(pdf_path):
    images = []
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            xobjects = page['/Resources']['/XObject'].getObject()

            for obj in xobjects:
                if xobjects[obj]['/Subtype'] == '/Image':
                    size = (xobjects[obj]['/Width'], xobjects[obj]['/Height'])
                    data = xobjects[obj].getData()
                    if xobjects[obj]['/ColorSpace'] == '/DeviceRGB':
                        mode = "RGB"
                    else:
                        mode = "P"

                    if xobjects[obj]['/Filter'] == '/FlateDecode':
                        img = np.frombuffer(data, dtype='uint8')
                        img = img.reshape(size[1], size[0], len(mode))
                        images.append(img)
                    elif xobjects[obj]['/Filter'] == '/DCTDecode':
                        img = open("temp.jpg", "wb")
                        img.write(data)
                        img.close()
                        img = cv2.imread("temp.jpg")
                        images.append(img)
    return images

In [15]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    return thresh

In [16]:
# Function to recognize handwritten numbers
def recognize_numbers(image):
    config = "--psm 6"  # Assume a single uniform block of text
    text = pytesseract.image_to_string(image, config=config)
    return text

In [17]:
pdf_path =  "Beispiel_Zeiten.pdf"

In [18]:
images = extract_images_from_pdf(pdf_path)
for idx, image in enumerate(images):
    preprocessed_image = preprocess_image(image)
    recognized_text = recognize_numbers(preprocessed_image)
    print(f"Page {idx + 1}: {recognized_text}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/sandradening/anaconda3/envs/Sandra_Coding/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/nd/xc3t3y5n57jcb763d6d9rksw0000gn/T/ipykernel_14888/4128599582.py", line 1, in <module>
    images = extract_images_from_pdf(pdf_path)
  File "/var/folders/nd/xc3t3y5n57jcb763d6d9rksw0000gn/T/ipykernel_14888/1734796440.py", line 6, in extract_images_from_pdf
    xobjects = page['/Resources']['/XObject'].getObject()
  File "/Users/sandradening/anaconda3/envs/Sandra_Coding/lib/python3.8/site-packages/PyPDF2/generic/_base.py", line 122, in getObject
  File "/Users/sandradening/anaconda3/envs/Sandra_Coding/lib/python3.8/site-packages/PyPDF2/_utils.py", line 369, in deprecation_with_replacement
    DeprecationWarnings
  File "/Users/sandradening/anaconda3/envs/Sandra_Coding/lib/python3.8/site-packages/PyPDF2/_utils.py", line 351, in deprecati

In [23]:


def extract_images_from_pdf(pdf_path):
    images = []
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            xobjects = page['/Resources']['/XObject']

            for obj in xobjects.keys():
                if xobjects[obj]['/Subtype'] == '/Image':
                    size = (xobjects[obj]['/Width'].get(), xobjects[obj]['/Height'].get())
                    data = xobjects[obj].get_object()._data
                    mode = "RGB" if xobjects[obj]['/ColorSpace'].get() == '/DeviceRGB' else "P"

                    if xobjects[obj]['/Filter'] == '/FlateDecode':
                        img = np.frombuffer(data, dtype='uint8')
                        img = img.reshape(size[1], size[0], len(mode))
                        images.append(img)
                    elif xobjects[obj]['/Filter'] == '/DCTDecode':
                        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                            tmp_file.write(data)
                            tmp_file.close()
                            img = cv2.imread(tmp_file.name)
                            images.append(img)
                            os.unlink(tmp_file.name)
    return images

# Function to preprocess image
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    return thresh

# Function to recognize handwritten numbers
def recognize_numbers(image):
    config = "--psm 6"  # Assume a single uniform block of text
    text = pytesseract.image_to_string(image, config=config)
    return text

# Main function
def main(pdf_path):
    try:
        images = extract_images_from_pdf(pdf_path)
        for idx, image in enumerate(images):
            preprocessed_image = preprocess_image(image)
            recognized_text = recognize_numbers(preprocessed_image)
            print(f"Page {idx + 1}: {recognized_text}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
pdf_path = "Beispiel_Zeiten.pdf"
main(pdf_path)

An error occurred: 'NumberObject' object has no attribute 'get'


In [29]:
import cv2
import numpy as np
import pytesseract
from PyPDF2 import PdfReader
import tempfile
import os

def extract_images_from_pdf(pdf_path):
    images = []
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            resources = page.get('/Resources', {})
            xobjects = resources.get('/XObject', {})
            for obj_key in xobjects.keys():
                obj = xobjects[obj_key]
                if obj['/Subtype'] == '/Image':
                    size = (obj['/Width'], obj['/Height'])
                    data = obj.get_object().get_data()
                    mode = "RGB"
                    if '/ColorSpace' in obj:
                        if obj['/ColorSpace'] == '/DeviceRGB':
                            mode = "RGB"
                        else:
                            mode = "P"

                    if obj['/Filter'] == '/FlateDecode':
                        img = np.frombuffer(data, dtype='uint8')
                        img = img.reshape(size[1], size[0], len(mode))
                        images.append(img)
                    elif obj['/Filter'] == '/DCTDecode':
                        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                            tmp_file.write(data)
                            tmp_file.close()
                            img = cv2.imread(tmp_file.name)
                            images.append(img)
                            os.unlink(tmp_file.name)
    return images

# Function to preprocess image
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    return thresh

# Function to recognize handwritten numbers
def recognize_numbers(image):
    config = "--psm 6"  # Assume a single uniform block of text
    text = pytesseract.image_to_string(image, config=config)
    return text

# Main function
def main(pdf_path):
    try:
        images = extract_images_from_pdf(pdf_path)
        for idx, image in enumerate(images):
            preprocessed_image = preprocess_image(image)
            recognized_text = recognize_numbers(preprocessed_image)
            print(f"Page {idx + 1}: {recognized_text}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage

# Example usage
pdf_path = "Test.pdf"
main(pdf_path)

An error occurred: 'IndirectObject' object has no attribute 'get'


In [ ]:
import pdfplumber

In [35]:
with pdfplumber.open("Test.pdf") as pdf:
    # Access PDF pages
    first_page = pdf.pages[0]
    #print(first_page)
    # Extract text from the first page
    text = first_page.extract_text()
    print(text)

Hallo
ich
bin
Sandra
12345781


In [36]:
text[0]

'H'

In [42]:
with pdfplumber.open("Beispiel_Zeiten.pdf") as pdf:
    # Access PDF pages
    first_page = pdf.pages[0]
    # Extract text from the first page
    text = first_page.extract_text()
    print(text)

In [43]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Apply OCR to the text extracted from the PDF
ocr_text = pytesseract.image_to_string(text, lang='eng')
ocr_text

TesseractNotFoundError: C:\Program Files\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information.

In [46]:
from pdf2image import convert_from_path
import cv2

In [51]:
pdf_path = "Test.pdf"

handwritten_regions = []

# Convert PDF pages to images
images = convert_from_path(pdf_path)

for idx, image in enumerate(images):
    # Convert PIL image to OpenCV format
    img_cv2 = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Convert image to grayscale
    gray = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to binarize the image
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours in the image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through contours and filter based on area or other criteria
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        # Add contour as a handwritten region if it meets certain criteria
        # You may need to adjust these criteria based on your documents
        if area > 1000:  # Example threshold for minimum area
            handwritten_regions.append((idx, (x, y, x+w, y+h)))  # Save page index and bounding box


In [52]:
print("Detected Handwritten Regions:", handwritten_regions)

Detected Handwritten Regions: [(0, (79, 78, 1575, 2223))]


In [56]:
text_in_regions = []

# Convert PDF pages to images
images = convert_from_path(pdf_path)
print(images)

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339 at 0x14FCCBB50>]


In [70]:
import os

# Define the path to the Tesseract executable
tesseract_path = '/Users/sandradening/anaconda3/envs/Sandra_Coding/bin/tesseract'  # Adjust this path as needed

# Check if the Tesseract executable exists
if os.path.exists(tesseract_path):
    print("Tesseract executable found at:", tesseract_path)
else:
    print("Tesseract executable not found. Please verify the path:", tesseract_path)

Tesseract executable not found. Please verify the path: /Users/sandradening/anaconda3/envs/Sandra_Coding/bin/tesseract


In [69]:
for idx, region in handwritten_regions:
    
    pytesseract.pytesseract.tesseract_cmd = r'/Users/sandradening/anaconda3/envs/Sandra_Coding/bin/pytesseract'
    
    page_image = images[idx]
    x0, y0, x1, y1 = region
    # Crop the handwritten region from the page image
    cropped_region = page_image.crop((x0, y0, x1, y1))
    # Perform OCR on the cropped region
    text = pytesseract.image_to_string(cropped_region)
    text_in_regions.append(text.strip())

text_in_regions

TesseractError: (2, 'Usage: pytesseract [-l lang] input_file')

In [ ]:
/Users/sandradening/anaconda3/envs/Sandra_Coding/bin/pytesseract